<a href="https://colab.research.google.com/github/miklaszewskigrzegorz/dw_matrix/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install  eli5


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook


In [22]:
cd "/content/drive/My Drive/Colab Notebooks"

/content/drive/My Drive/Colab Notebooks


In [23]:
cd dw_matrix/

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [25]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageurls', 'isbn', 'keys', 'manufacturer',
       'manufacturernumber', 'merchants', 'name', 'prices_amountmin',
       'prices_amountmax', 'prices_availability', 'prices_color',
       'prices_condition', 'prices_count', 'prices_currency',
       'prices_dateadded', 'prices_dateseen', 'prices_flavor', 'prices_issale',
       'prices_merchant', 'prices_offer', 'prices_returnpolicy',
       'prices_shipping', 'prices_size', 'prices_source', 'prices_sourceurls',
       'prices_warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceurls', 'upc', 'vin', 'websiteids', 'weight'],
      dtype='object')

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[ feats ].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model,X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
# df['brand_cat'] = df['brand'].factorize()[0]
# future engineering
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]

In [28]:
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

usprawniamy jakość modelu po zsatosowaniu - Random Forest

In [29]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

Feature Engineering -  to lepszego poznania danych - operacje na danych
- wyżej żeby brand było przrównanne wszytskie z małej litery

In [30]:
df.head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,reviews,sizes,skus,sourceurls,upc,vin,websiteids,weight,brand_cat
0,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
1,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
2,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
3,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,50.31,50.31,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2015-11-30T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
4,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""

In [31]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [32]:
test = {'key': 'value'}
test['key']

str(test)

"{'key': 'value'}"

In [33]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'
#literal_eval(str_dict) [0]['key']
#literal_eval(str_dict) [0]['value']
literal_eval(str_dict) [0]['value'][0]

'Men'

In [0]:
#def parse_features(x):
 # if str(x) == 'nan': return [] # jeżeli wartość jest pusta to pusta lista
 # return literal_eval(x)
#replace('\\", '"'))
#  df['features_parsed'] = df['features'].map(parse_features)

In [0]:
 def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict
  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()
    output_dict[key] = value
  return output_dict
df['features_parsed'] = df['features'].map(parse_features)

In [36]:
df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [37]:
keys = set()

df['features_parsed'].map( lambda x: keys.update(x.keys()))

len(keys)


476

In [38]:
def get_name_feat(key):
  return 'feat_' + key
  
for key in tqdm_notebook(keys):
  #print(key)
  #break
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats:feats[key]  if key in feats else np.nan)

In [39]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_stylenumber', 'feat_charger included', 'feat_sub type',
       'feat_waist', 'feat_lining material', 'feat_bag size range',
       'feat_material detail-1', 'feat_crystal', 'feat_chest size',
       'feat_contains batteries'],
      dtype='object', length=526)

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100

In [41]:
{k:v for k,v in keys_stat.items() if v > 30}
#output tylko gender występuje w 50% parametrów

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
#normalizacja jednostek wagi 
def normalize_to_gram(line):
    if str(line) == 'nan' : return -1
    

    value, unit = line.lower().split(' ')
    value = float(value)

    if unit == 'lbs' or unit == 'lb' or unit == 'pounds':
        value *= 453.592
    elif 'kg' == unit:
        value *= 1000
    elif 'ounces' == unit:
        value *= 28.3495
    elif 'g' == unit:
        pass
    else:
        print("ERROR: ", unit)
    return value

df['weight_norm'] = df['weight'].map(normalize_to_gram)

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer_cat'] = df['feat_manufacturer'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_size_cat'] = df['feat_size'].factorize()[0]
df['feat_weight_norm_cat'] = df['feat_weight'].factorize()[0]

#wersja z usprawnionym normlaizacją dla wszystkich _cat
for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]

In [86]:
run_model(['feat_brand_cat'])

(-61.79232381882688, 3.7460167781236606)

In [87]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())
df[df.brand == df.feat_brand].shape

(8846, 1004)

In [0]:
feats = ['']

In [89]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

(-57.280639556929074, 4.166458730913197)

In [90]:
feats_cat = [x for x in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'feat_recommended location',
 'feat_catalog',
 'feat_location - country',
 'feat_fabrication',
 'feat_location - city/state',
 'feat_certifications and listings',
 'feat_shoe category',
 'feat_clothing category',
 'feat_multi pack indicator',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_size_cat',
 'feat_age segment_cat',
 'feat_seller inventory number_cat',
 'feat_fastener_cat',
 'feat_heart rate monitor_cat',
 'feat_athlete_cat',
 'feat_case diameter_cat',
 'feat_style number_cat',
 'feat_hammer loop_cat',
 'feat_era_cat',
 'feat_leg opening:_cat',
 'feat_pattern_cat',
 'feat_number of pieces_cat',
 'feat_����������������_cat',
 'feat_hardsided or softsided_cat',
 'feat_vehicle type_cat',
 'feat_dial markers_cat',
 'feat_is portable_cat',
 'feat_country of manufacturer_cat',
 'feat_features_cat',
 'feat_style_cat',
 'feat_style:_cat',
 'feat_product id_cat',
 'feat_rel

In [0]:
feats = ['brand_cat','feat_brand_cat','feat_gender_cat','feat_material_cat','feat_size_cat', 'feat_weight_norm_cat']
#feats += feats_cat
#feats = list(set(feats)) # unikalna lista
model = RandomForestRegressor(max_depth=5, n_estimators=100)

result = run_model(feats, model)

In [101]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

print(result)
perm = PermutationImportance(m,random_state=1).fit(X,y);
eli5.show_weights(perm, feature_names=feats)



(-57.00357565753129, 4.23338437860591)


Weight,Feature
0.2579 ± 0.0101,brand_cat
0.0988 ± 0.0074,feat_material_cat
0.0445 ± 0.0042,feat_gender_cat
0.0208 ± 0.0012,feat_brand_cat
0.0136 ± 0.0010,feat_weight_norm_cat
0.0011 ± 0.0004,feat_size_cat


#output

#Weight	Feature
#0.2578 ± 0.0112	brand_cat
#0.1058 ± 0.0094	feat_material_cat
#0.0423 ± 0.0043	feat_gender_cat
#0.0213 ± 0.0011	feat_brand_cat
#0.0077 ± 0.0004	feat_color_cat - usuwamy tą cechę jako taką która nie wpływa na wynik
#0.0002 ± 0.0000	feat_manufacturer_cat - usuwamy tą cechę jako taką która nie wpływa na wynik

0.2595 ± 0.0080	brand_cat - widać że Brand znacząco wpływa na wynik - dlatego trzeba bliżej mu się przyjrzeć. 

In [69]:
df['brand'].value_counts(normalize=True)

nike                0.097210
puma                0.033315
ralph lauren        0.028775
vans                0.021116
new balance         0.020295
                      ...   
nba                 0.000055
rlx ralph lauren    0.000055
authentic icon      0.000055
carroll leather     0.000055
ryan seacrest       0.000055
Name: brand, Length: 1732, dtype: float64

In [70]:
df[df['brand'] == 'nike'].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [71]:
df[df['brand'] == 'nike'].features_parsed.sample(5).values

array([{'style': 'athletic', 'condition': 'new with tags'}, {},
       {'season': 'all-season', 'material': 'leather', 'gender': 'men', 'shoe size': '8', 'color': 'gray', 'manufacturer part number': '631685 017', 'brand': 'nike', 'age group': 'adult'},
       {'color': 'green yellow red dark gray'},
       {'material': 'mesh', 'gender': 'men', 'shoe category': "men's shoes", 'assembled product dimensions (l x w x h)': '12.00 x 8.00 x 6.00 inches', 'fabric content': 'mesh', 'color': 'wolf grey/racer blue/blk/wht', 'model': '844514 004', 'shoe closure': 'lace-up', 'manufacturer part number': '844514 004', 'brand': 'nike', 'shoe width': 'd(m)'}],
      dtype=object)

In [72]:
df['feat_age group'].value_counts()

adult               4563
men                  350
child                 77
men's                 33
unisex                 6
infant                 4
mens                   4
toddler                4
boys'                  3
women ,�� unisex       2
women                  2
youth                  2
men||women             2
12 up                  1
adult ,�� teen         1
Name: feat_age group, dtype: int64

In [0]:
!git add matrix_one/day5.ipynb


In [106]:
!git commit -m "DAY 5 - Read Men's Shoe Prices dataset from data.world"



*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@82a3e368cd0c.(none)')


In [0]:
!git config --global user.email "miklaszewskigrzegorz@gmail.com"
!git config --global user.name "Grzegorz Mikłaszewski"


In [108]:
!git commit -m "DAY 5 - Read Men's Shoe Prices dataset from data.world"

[master 7af4240] DAY 5 - Read Men's Shoe Prices dataset from data.world
 1 file changed, 1 insertion(+)
 create mode 100644 matrix_one/day5.ipynb


In [110]:
!git push -u origin master

To https://github.com/miklaszewskigrzegorz/dw_matrix.git
 ! [rejected]        master -> master (non-fast-forward)
error: failed to push some refs to 'https://906bab3a1f3626bf82055c14725d558af31c90e7@github.com/miklaszewskigrzegorz/dw_matrix.git'
hint: Updates were rejected because the tip of your current branch is behind
hint: its remote counterpart. Integrate the remote changes (e.g.
hint: 'git pull ...') before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.


In [112]:
!git pull origin master 
!git add  matrix_one/day5.ipynb 
!git commit -m "Read Men's Shoe Prices dataset from data.world"
!git push origin master

From https://github.com/miklaszewskigrzegorz/dw_matrix
 * branch            master     -> FETCH_HEAD
error: Your local changes to the following files would be overwritten by merge:
	matrix_one/day4.ipynb
Please commit your changes or stash them before you merge.
Aborting
[master 4e5c426] Read Men's Shoe Prices dataset from data.world
 1 file changed, 1 insertion(+), 1 deletion(-)
To https://github.com/miklaszewskigrzegorz/dw_matrix.git
 ! [rejected]        master -> master (non-fast-forward)
error: failed to push some refs to 'https://906bab3a1f3626bf82055c14725d558af31c90e7@github.com/miklaszewskigrzegorz/dw_matrix.git'
hint: Updates were rejected because the tip of your current branch is behind
hint: its remote counterpart. Integrate the remote changes (e.g.
hint: 'git pull ...') before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.
